In [ ]:
#| default_exp utils

# utils
> Assorted utility functions

In [ ]:
#| exporti 

import inspect

## Delegation
> A decorator that allows us to share parameters from one function to another

This function is adapted from [this brilliant blog post](https://www.fast.ai/posts/2019-08-06-delegation.html#solving-the-problem-with-delegated-composition) by Jeremy Howard that outlines the advantages of delegation in Python. We will use this decorator to adapt several Pandas functions for our use cases. 

In [ ]:

#|export

def delegates(to=None, keep=False):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to,f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

### Delegation example

In [ ]:
from nbdev.showdoc import show_doc

Example: Create a fuction called `break_the_ice` that delegates to another function called `say_hi`:

In [ ]:

def say_hi(name:str=None):
    return f"Hi! my name is {name}."
    

@delegates(say_hi)
def break_the_ice(fun_fact:str,**kwargs):
    
    hi = say_hi(name=kwargs['name'])
    intro = hi + ' ' + fun_fact
    return intro
    

show_doc(break_the_ice)

---

### break_the_ice

>      break_the_ice (fun_fact:str, name:str=None)

In [ ]:
break_the_ice(name='Charlie',fun_fact='I like music')

'Hi! my name is Charlie. I like music'

:::{.callout-warning}
The function that you're delegating to (i.e. the function within the call of the decorator) **must** have default values for the parameters that you want to pass to the decorated function 
:::

In [ ]:
def first_function(
    this_wont_show:str,
    this_will_show:str=None
):
    pass 

@delegates(first_function)
def second_function(
    another_param:str,
    **kwargs
):
    pass

show_doc(second_function)

---

### second_function

>      second_function (another_param:str, this_will_show:str=None)

In [ ]:
#| hide
!nbdev_export 